In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mtcnn import MTCNN
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Veri Yolu
DATASET_PATH = r"C:\Users\Admin\Desktop\TEZ\Deepfake"
OUTPUT_PATH = r"C:\Users\Admin\Desktop\TEZ\Deepfake\ProcessedFaces"
TEST_VIDEO_LIST = "test_videos.txt"
FRAME_INTERVAL = 10  # Her 10 karede bir frame al
IMAGE_SIZE = (128, 128)  # Model için kullanılacak boyut

detector = MTCNN()

def extract_faces(video_path, output_folder):
    print(f"Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success = True
    
    while success:
        success, frame = cap.read()
        if not success:
            break
        
        if frame_count % FRAME_INTERVAL == 0:
            faces = detector.detect_faces(frame)
            for i, face in enumerate(faces):
                x, y, width, height = face['box']
                face_img = frame[y:y+height, x:x+width]
                face_img = cv2.resize(face_img, IMAGE_SIZE)
                save_path = os.path.join(output_folder, f"{os.path.basename(video_path)}_frame{frame_count}_face{i}.jpg")
                cv2.imwrite(save_path, face_img)
                #print(f"Saved: {save_path}")
        
        frame_count += 1
    
    cap.release()

def process_videos():
    start_time = time.time()
    print("Starting face extraction from videos...")
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    
    test_videos = []
    
    for category in ["Celeb-real", "Youtube-real", "Celeb-synthesis"]:
        input_folder = os.path.join(DATASET_PATH, category)
        output_folder = os.path.join(OUTPUT_PATH, category)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        video_files = os.listdir(input_folder)
        test_videos.extend([os.path.join(input_folder, v) for v in video_files])
        
        for video_file in video_files:
            video_path = os.path.join(input_folder, video_file)
            extract_faces(video_path, output_folder)
    
    with open(TEST_VIDEO_LIST, "w") as f:
        for video in test_videos:
            f.write(video + "\n")
    
    end_time = time.time()
    print(f"Yüz çıkarma işlemi tamamlandı! Süre: {end_time - start_time:.2f} saniye")
    print(f"Test videoları kaydedildi: {TEST_VIDEO_LIST}")

def build_model(model_type='mobilenet'):
    print(f"Building {model_type} model...")
    if model_type == 'mobilenet':
        base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
    elif model_type == 'efficientnet':
        base_model = EfficientNetB0(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
    else:
        raise ValueError("Invalid model type. Choose 'mobilenet' or 'efficientnet'.")
    
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(model_type='mobilenet'):
    start_time = time.time()
    print("Starting model training...")
    datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
    train_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='training')
    val_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='validation')
    
    model = build_model(model_type)
    history = model.fit(train_generator, validation_data=val_generator, epochs=10)
    model.save(f"deepfake_detector_{model_type}.h5")
    
    end_time = time.time()
    print(f"Model eğitimi tamamlandı ve deepfake_detector_{model_type}.h5 olarak kaydedildi! Eğitim süresi: {end_time - start_time:.2f} saniye")

def evaluate_model(model_type='mobilenet'):
    print(f"Evaluating {model_type} model...")
    model = load_model(f"deepfake_detector_{model_type}.h5")
    datagen = ImageDataGenerator(rescale=1./255)
    test_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='validation', shuffle=False)
    
    predictions = model.predict(test_generator)
    y_pred = (predictions >= 0.5).astype(int)
    y_true = test_generator.classes
    
    print(f"Classification Report for {model_type}:")
    print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
    print(f"Confusion Matrix for {model_type}:")
    print(confusion_matrix(y_true, y_pred))

# Çalıştır
if __name__ == "__main__":
    process_videos()
    train_model(model_type='mobilenet')
    evaluate_model(model_type='mobilenet')
    train_model(model_type='efficientnet')
    evaluate_model(model_type='efficientnet')


Starting face extraction from videos...
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0000.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0001.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0002.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0003.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0004.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0005.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0006.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0007.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0008.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id0_0009.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id10_0000.mp4
Processing video: C:\Users\Admin\Desktop\TEZ\Deepfake\Celeb-real\id10_0001.mp4
Processing video: C:\U

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1241s 189ms/step - accuracy: 0.8564 - loss: -148.4584 - val_accuracy: 0.8574 - val_loss: -104.8851
Epoch 2/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1058s 162ms/step - accuracy: 0.8587 - loss: -1973.2831 - val_accuracy: 0.8569 - val_loss: -370.0075
Epoch 3/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1052s 161ms/step - accuracy: 0.8587 - loss: -5816.4082 - val_accuracy: 0.8575 - val_loss: -788.0582
Epoch 4/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1034s 158ms/step - accuracy: 0.8583 - loss: -12025.7119 - val_accuracy: 0.8569 - val_loss: -1483.2144
Epoch 5/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1031s 157ms/step - accuracy: 0.8563 - loss: -19383.8027 - val_accuracy: 0.8556 - val_loss: -2468.5964
Epoch 6/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1034s 158ms/step - accuracy: 0.8576 - loss: -28833.9668 - val_accuracy: 0.8527 - val_loss: -3625.8521
Epoch 7/10
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 1050s 160ms/step - accuracy: 0.8574 - loss: -40848.9258 - val_accuracy: 0.8520 - val_loss: -5025.6919

Model eğitimi tamamlandı ve deepfake_detector_mobilenet.h5 olarak kaydedildi! Eğitim süresi: 10710.23 saniye
Evaluating mobilenet model...


Found 0 images belonging to 3 classes.


ValueError: The PyDataset has length 0

In [7]:
import os
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

#GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"1. Aşama - Cihaz kontrolü: Kullanılan cihaz: {device}")


# Modelin Yolu
model_path = "deepfake_detector_mobilenet.h5"

# Veri Yolu
OUTPUT_PATH = "ProcessedFaces"
IMAGE_SIZE = (128, 128)
batch_size = 32

# Modeli Yükle
model = load_model(model_path)
print(f"Model {model_path} başarıyla yüklendi!")

# Test Verisini Hazırla
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory(
    OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=batch_size, class_mode='binary', shuffle=False
)

# Tahminleri Al
predictions = model.predict(test_generator)
y_pred = (predictions >= 0.5).astype(int)  # 0.5 eşik değerine göre sınıflandırma
y_true = test_generator.classes  # Gerçek etiketler

# Performans Metriklerini Yazdır
print("\n📊 **MobileNet Modeli İçin Sınıflandırma Raporu:**")
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))

# Karışıklık Matrisi (Confusion Matrix)
print("\n🌀 **Confusion Matrix:**")
print(confusion_matrix(y_true, y_pred))


1. Aşama - Cihaz kontrolü: Kullanılan cihaz: cuda


Model deepfake_detector_mobilenet.h5 başarıyla yüklendi!
Found 261976 images belonging to 3 classes.


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1043/8187 ━━━━━━━━━━━━━━━━━━━━ 13:17 112ms/step

KeyboardInterrupt: 

In [1]:
import os
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# GPU Kullanımını Zorunlu Tut
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"✅ GPU Kullanılıyor: {physical_devices[0]}")
    device_name = "/GPU:0"
else:
    print("⚠ GPU bulunamadı, işlemci (CPU) kullanılacak!")
    device_name = "/CPU:0"

# PyTorch için GPU Kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"1. Aşama - Cihaz kontrolü: Kullanılan cihaz: {device}")

# Modelin Yolu
model_path = "deepfake_detector_mobilenet.h5"

# Veri Yolu
OUTPUT_PATH = "ProcessedFaces"
IMAGE_SIZE = (128, 128)
batch_size = 32

# Modeli Yükle ve GPU'ya Taşı
with tf.device(device_name):
    model = load_model(model_path)
print(f"✅ Model {model_path} başarıyla yüklendi ve GPU'ya taşındı!")

# Test Verisini Hazırla
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory(
    OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=batch_size, class_mode='binary', shuffle=False
)

# GPU Kullanarak Tahmin Yap
with tf.device(device_name):
    predictions = model.predict(test_generator)

# Tahminleri 0.5 eşik değerine göre sınıflandır
y_pred = (predictions >= 0.5).astype(int)
y_true = test_generator.classes  # Gerçek etiketler

# Performans Metriklerini Yazdır
print("\n📊 **MobileNet Modeli İçin Sınıflandırma Raporu:**")
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))

# Karışıklık Matrisi (Confusion Matrix)
print("\n🌀 **Confusion Matrix:**")
print(confusion_matrix(y_true, y_pred))


⚠ GPU bulunamadı, işlemci (CPU) kullanılacak!
1. Aşama - Cihaz kontrolü: Kullanılan cihaz: cuda


✅ Model deepfake_detector_mobilenet.h5 başarıyla yüklendi ve GPU'ya taşındı!
Found 261976 images belonging to 3 classes.


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 313/8187 ━━━━━━━━━━━━━━━━━━━━ 14:56 114ms/step

KeyboardInterrupt: 

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Veri Yolu
OUTPUT_PATH = r"C:\Users\Admin\Desktop\TEZ\Deepfake\ProcessedFaces"
IMAGE_SIZE = (128, 128)  # Model için kullanılacak boyut

def build_model():
    print("Building EfficientNet model...")
    base_model = EfficientNetB0(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
    base_model.trainable = False
    
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model():
    start_time = time.time()
    print("Starting EfficientNet model training...")
    datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
    train_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='training')
    val_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='validation')
    
    model = build_model()
    history = model.fit(train_generator, validation_data=val_generator, epochs=10)
    model.save("deepfake_detector_efficientnet.h5")
    
    end_time = time.time()
    print(f"EfficientNet modeli eğitildi ve deepfake_detector_efficientnet.h5 olarak kaydedildi! Eğitim süresi: {end_time - start_time:.2f} saniye")

def evaluate_model():
    print("Evaluating EfficientNet model...")
    model = load_model("deepfake_detector_efficientnet.h5")
    datagen = ImageDataGenerator(rescale=1./255)
    test_generator = datagen.flow_from_directory(
        OUTPUT_PATH, target_size=IMAGE_SIZE, batch_size=32, class_mode='binary', subset='validation', shuffle=False)
    
    predictions = model.predict(test_generator)
    y_pred = (predictions >= 0.5).astype(int)
    y_true = test_generator.classes
    
    print("Classification Report for EfficientNet:")
    print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))
    print("Confusion Matrix for EfficientNet:")
    print(confusion_matrix(y_true, y_pred))

# Çalıştır
if __name__ == "__main__":
    train_model()
    evaluate_model()


Starting EfficientNet model training...
Found 209581 images belonging to 3 classes.
Found 52395 images belonging to 3 classes.
Building EfficientNet model...
Epoch 1/10
  76/6550 ━━━━━━━━━━━━━━━━━━━━ 18:06 168ms/step - accuracy: 0.8448 - loss: 0.9164